In [142]:
import pandas as pd
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as soup

In [261]:
kospi_df = pd.read_csv("/Users/nayo/Downloads/data_0436_20220224.csv", names= ["code", "name", "market", "industry", "close", "contrast", "change", "market_cap"], encoding = "cp949", header = 0)
kosdaq_df = pd.read_csv("/Users/nayo/Downloads/data_0535_20220224.csv", names= ["code", "name", "market", "industry", "close", "contrast", "change", "market_cap"], encoding = "cp949", header = 0)
stock_df = pd.concat([kospi_df, kosdaq_df]).reset_index(drop = True)
stock_df

,code,name,market,industry,close,contrast,change,market_cap
0,095570,AJ네트웍스,KOSPI,서비스업,5460,40,0.74,255649730700
1,006840,AK홀딩스,KOSPI,기타금융,21600,-400,-1.82,286147317600
2,027410,BGF,KOSPI,기타금융,5070,-20,-0.39,485284130370
3,282330,BGF리테일,KOSPI,유통업,163000,1000,0.62,2817276678000
4,138930,BNK금융지주,KOSPI,기타금융,7980,10,0.13,2600963263080
...,...,...,...,...,...,...,...,...
2480,024060,흥구석유,KOSDAQ,유통,8430,120,1.44,126450000000
2481,010240,흥국,KOSDAQ,기계·장비,6560,-10,-0.15,80836885760
2482,189980,흥국에프엔비,KOSDAQ,음식료·담배,3965,35,0.89,158591162015
2483,037440,희림,KOSDAQ,기타서비스,10250,150,1.49,142705368750


In [275]:
stock_code = stock_df["code"].to_list()

In [276]:
def get_summary(text):
    temp = str()
    for tag in range(len(text)):
        temp = temp + text[tag].get_text() + os.linesep
    return temp

In [277]:
def concat_business_data(df):

    i = 1
    business_lst = list()
    stock_code = df["code"].to_list()
    for code in tqdm(stock_code):
        url = f"https://finance.naver.com/item/main.naver?code={code}"
        response = requests.get(url)
        html = response.text
        tagged_text = soup(html, "html.parser").select(".summary_info > p")


        stock_summary = get_summary(tagged_text)
        business_lst.append(stock_summary)
        if i % 100 == 0:
            time.sleep(20)
        i += 1

    new_df = pd.concat([df, pd.Series(business_lst)], axis = 1)
    new_df.columns = ["code", "name", "market", "industry", "close", "contrast", "change", "market_cap", "business"]
    
    return new_df


stock_df = concat_business_data(stock_df)
# stock_df.to_csv("stock_df.csv")
stock_df


100%|██████████| 2485/2485 [17:53<00:00,  2.31it/s]  


In [285]:
stock_df

,code,name,market,industry,close,contrast,change,market_cap,business
0,095570,AJ네트웍스,KOSPI,서비스업,5460,40,0.74,255649730700,"2000년 설립된 동사는 파렛트, OA기기, 건설장비 장단기 대여·판매업을 주요 사..."
1,006840,AK홀딩스,KOSPI,기타금융,21600,-400,-1.82,286147317600,2012년 9월 인적분할을 통해 제조사업부문을 신설회사 애경유화에 승계하고 지주회사...
2,027410,BGF,KOSPI,기타금융,5070,-20,-0.39,485284130370,"동사는 1990년 편의점 사업을 시작하였으며, 편의점 체인화 사업을 목적으로 199..."
3,282330,BGF리테일,KOSPI,유통업,163000,1000,0.62,2817276678000,2017년 인적분할하여 설립된 신설법인으로 2017년 12월 유가증권시장에 재상장되...
4,138930,BNK금융지주,KOSPI,기타금융,7980,10,0.13,2600963263080,"동사는 2011년 부산은행, BNK투자증권, BNK신용정보, BNK캐피탈이 공동으로..."
...,...,...,...,...,...,...,...,...,...
2480,024060,흥구석유,KOSDAQ,유통,8430,120,1.44,126450000000,동사는 1966년 12월 16일에 설립되어 1994년 11월 30일자로 코스닥시장에...
2481,010240,흥국,KOSDAQ,기계·장비,6560,-10,-0.15,80836885760,"동사는 1974년 10월 17일에 철강단조 등을 주 영업목적으로 설립되었으며, 20..."
2482,189980,흥국에프엔비,KOSDAQ,음식료·담배,3965,35,0.89,158591162015,"2008년 3월 설립되어 청량음료, 영양식품, 과즙음료, 과실, 기타 식품첨가물 등..."
2483,037440,희림,KOSDAQ,기타서비스,10250,150,1.49,142705368750,동사는 1970년 1월 10일 희림건축설계사무소로 설립되어 2000년 2월 3일자로...
